# Creating records 
## Pipeline 2.0
##### ASTROMER dev team

*JAN 17 2023*

In [9]:
cd /home

/home


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import os

from src.data.record import DataPipeline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
METAPATH = './data/raw_data/alcock/metadata.csv'
# LCDIR = 'LCs/' 
LCDIR = './data/raw_data/alcock/LCs/'

In [12]:
metadata = pd.read_csv(METAPATH)

In [13]:
metadata['Class'] = pd.Categorical(metadata['Class'])
metadata['Label'] = metadata['Class'].cat.codes
metadata['Path'] = metadata['Path'].apply(lambda x: os.path.join(LCDIR, x)) 

In [14]:
metadata.sample()

,ID,Class,Path,Band,Label
12450,6.6329.656,RRab,./data/raw_data/alcock/LCs/6.6329.656.dat,1.0,4


### Using DataPipeline class

In [7]:
pipeline = DataPipeline(metadata=metadata, 
                        context_features=['ID', 'Label', 'Class'],
                        sequential_features=['mjd', 'mag'],)

[INFO] 21444 samples loaded


In [8]:
%%time
var = pipeline.run(n_jobs=8)

[INFO] Processing data...
[INFO] Writing records...


  0%|                                                              | 0/21444 [00:00<?, ?it/s]2023-01-17 15:12:58.726941: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (34)
2023-01-17 15:12:58.726982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (e1977df45e7c): /proc/driver/nvidia/version does not exist
2023-01-17 15:12:58.727304: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|████████████████████████████████████████████████| 21444/21444 [00:06<00:00, 3192.22it/s]

CPU times: user 22.3 s, sys: 1.24 s, total: 23.5 s
Wall time: 26.6 s


### Customize the preprocessing method of DataPipeline

You must keep the same parameters of the method i.e., `row, context_features, sequential_features`. 

Also the **output** should be tuple containing the lightcurve (`pd.DataFrame`) and the context values (`dict`)


To modify the `process_sample` method we need to create a new class (`MyPipeline`) that inherits from `DataPipeline` 

In [20]:
class MyPipeline(DataPipeline):
    @staticmethod
    def process_sample(row, context_features, sequential_features):
        observations = pd.read_csv(row['Path'])
        observations.columns = ['mjd', 'mag', 'errmag']
        observations = observations.dropna()
        observations.sort_values('mjd')
        observations[observations['errmag'] < 1]
        context_features_values = row[context_features]
        return observations, context_features_values

Next steps are the same as using the original `DataPipeline` class

In [21]:
custom_pipeline = MyPipeline(metadata=metadata, 
                             context_features=['ID', 'Label', 'Class'],
                             sequential_features=['mjd', 'mag'])

[INFO] 21444 samples loaded


In [22]:
%%time
var = custom_pipeline.run(n_jobs=8)

[INFO] Processing data...
[INFO] Writing records...


100%|████████████████████████████████████████████████| 21444/21444 [00:08<00:00, 2488.90it/s]

CPU times: user 22.1 s, sys: 1.27 s, total: 23.4 s
Wall time: 24.9 s
